Install Dependencies

In [ ]:
!pip install transformers datasets peft

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00


Import Libraries

In [ ]:
import torch
from datasets import load_dataset
from transformers import GPTNeoForCausalLM, GPT2Tokenizer, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from torch.nn import CrossEntropyLoss
from peft import get_peft_model, LoraConfig, TaskType

Load Model and Tokenizer

In [ ]:
model_name = "EleutherAI/gpt-neo-1.3B"
model = GPTNeoForCausalLM.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# Define LoRA configuration
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Prepare Dataset

In [ ]:
# Load dataset
data = load_dataset('json', data_files='/content/data.json')

# Ambil bagian train dari dataset
train_data = data['train']

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

# Tambahkan pad_token jika belum ada
tokenizer.pad_token = tokenizer.eos_token

# Preprocess function
def preprocess_function(examples):
    # Gabungkan prompt dan response
    full_texts = [f"{prompt} {response}" for prompt, response in zip(examples['prompt'], examples['response'])]

    # Tokenize the full texts
    return tokenizer(full_texts, truncation=True, max_length=256, padding='max_length')

# Tokenize data
tokenized_data = train_data.map(preprocess_function, batched=True)

# Data collator for dynamic padding
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

Training Arguments

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="no",
    learning_rate=5e-4,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    weight_decay=0.001,
    save_total_limit=2,
    fp16=True,
    gradient_accumulation_steps=2,  # Coba kurangi ini
    logging_dir='./logs',
    logging_steps=10,
    dataloader_num_workers=2,
    report_to="none",
)


Trainer Setup

In [ ]:
# Custom Trainer class to handle loss calculation
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs['input_ids']
        outputs = model(**inputs)
        logits = outputs.logits

        # Shift so that tokens < n predict n
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()

        # Flatten the tokens
        loss_fct = CrossEntropyLoss()
        loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
        return (loss, outputs) if return_outputs else loss

# Initialize the custom trainer
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Start Training!

In [ ]:
trainer.train()

Step,Training Loss
10,4.080500
20,4.374700
30,4.499300
40,4.592100
50,4.633000


TrainOutput(global_step=58, training_loss=4.391038039634967, metrics={'train_runtime': 11.5904, 'train_samples_per_second': 5.004, 'train_steps_per_second': 5.004, 'total_flos': 107798977314816.0, 'train_loss': 4.391038039634967, 'epoch': 1.0})

Save the Model

In [ ]:
model.save_pretrained("arifian-ai")
tokenizer.save_pretrained("arifian-ai")

('arifian-ai/tokenizer_config.json',
 'arifian-ai/special_tokens_map.json',
 'arifian-ai/vocab.json',
 'arifian-ai/merges.txt',
 'arifian-ai/added_tokens.json')

Testing the Fine-Tuned Model

In [ ]:
from transformers import pipeline

generator = pipeline('text-generation', model="arifian-ai", tokenizer=tokenizer, device=0)

prompt = "Nama saya"
result = generator(prompt, max_length=50, num_return_sequences=1)
print(result[0]['generated_text'])


OutOfMemoryError: CUDA out of memory. Tried to allocate 394.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 27.06 MiB is free. Process 4741 has 14.72 GiB memory in use. Of the allocated memory 14.38 GiB is allocated by PyTorch, and 208.40 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)